<h1 style="text-align: center;">Schedule Optimization Effort + Simulations</h1>

## Necessary Libraries

In [1]:
import pandas as pd
import numpy as np 
import itertools


## Loading in the Data

In [2]:
runs_master = pd.read_excel('files/2025 Big West Conference Runs.xlsx', sheet_name='Conference RunsIP')
runs_master

,Team,Conf R/GP,Conf RA/GP,Conf R/IP,Conf RA/IP,Conf ATT,Conf DEF
0,Cal Bap,6.178490,6.137300,0.686499,0.681922,1.000392,1.011012
1,Cal Poly,7.214724,5.042945,0.801636,0.560327,1.168174,0.830736
2,Bakersfield,4.270510,6.086475,0.474501,0.676275,0.691461,1.002640
3,Fullerton,6.000000,5.823529,0.666667,0.647059,0.971492,0.959324
4,CSUN,6.236364,7.690909,0.692929,0.854545,1.009763,1.266942
5,LBSU,5.521472,6.478528,0.613497,0.719836,0.894011,1.067223
6,Sacramento State,5.200000,5.040000,0.577778,0.560000,0.841960,0.830251
7,UCI,8.688391,5.389002,0.965377,0.598778,1.406784,0.887743
8,UCR,4.115086,8.020218,0.457232,0.891135,0.666295,1.321190
9,UCSD,7.302632,6.039474,0.811404,0.671053,1.182408,0.994897


## Simulation

### Set Home Field Advantage

In [3]:
hfa = {
    'Cal Bap': 0.23,
    'Cal Poly': 0.18,
    'UCI': 0.1, 
    'UCSB': 0.22,
    'UCR': 0.26,
    'LBSU': 0.22,
    'CSUN': 0.2,
    'UCSD': 0.24,
    'Fullerton': 0.19,
    'Utah Valley': 0.19,
    'Sacramento State': 0.17,
    'Bakersfield': 0.21
}

### Add Expected Runs

In [4]:
# Find the average runs scored by any team in the conference
average_runs = runs_master[runs_master['Team'] == 'Average']['Conf R/GP'].iloc[0]

def find_expected_runs(opponent):
    
    if opponent == 'Average':
        return np.nan 
    if opponent == 'UCSD': 
        return np.nan
    
    #Find our offensive rating
    att = runs_master[runs_master['Team'] == 'UCSD']['Conf ATT'].iloc[0]
    
    #Find opponents defensive rating
    defense = runs_master[runs_master['Team'] == opponent]['Conf DEF'].iloc[0]
    
    #Calculate expected runs
    xruns = (att * defense) * average_runs * np.sqrt(1 + hfa['UCSD'])
    
    return round(xruns, 2)


def find_opp_expected_runs(opponent):
    
    if opponent == 'Average':
        return np.nan
    
    if opponent == 'UCSD':
        return np.nan
    
    att = runs_master[runs_master['Team'] == opponent]['Conf ATT'].iloc[0]
    
    defense = runs_master[runs_master['Team'] == 'UCSD']['Conf DEF'].iloc[0]
    
    xruns = (att * defense) * average_runs * np.sqrt(1 + hfa[opponent])
    
    return round(xruns, 2)
    

runs_master['ucsd_xruns'] = runs_master['Team'].apply(find_expected_runs) 
runs_master['x_runs'] = runs_master['Team'].apply(find_opp_expected_runs)

runs_master
    

,Team,Conf R/GP,Conf RA/GP,Conf R/IP,Conf RA/IP,Conf ATT,Conf DEF,ucsd_xruns,x_runs
0,Cal Bap,6.178490,6.137300,0.686499,0.681922,1.000392,1.011012,8.22,6.82
1,Cal Poly,7.214724,5.042945,0.801636,0.560327,1.168174,0.830736,6.76,7.80
2,Bakersfield,4.270510,6.086475,0.474501,0.676275,0.691461,1.002640,8.15,4.67
3,Fullerton,6.000000,5.823529,0.666667,0.647059,0.971492,0.959324,7.80,6.51
4,CSUN,6.236364,7.690909,0.692929,0.854545,1.009763,1.266942,10.30,6.80
5,LBSU,5.521472,6.478528,0.613497,0.719836,0.894011,1.067223,8.68,6.07
6,Sacramento State,5.200000,5.040000,0.577778,0.560000,0.841960,0.830251,6.75,5.60
7,UCI,8.688391,5.389002,0.965377,0.598778,1.406784,0.887743,7.22,9.07
8,UCR,4.115086,8.020218,0.457232,0.891135,0.666295,1.321190,10.74,4.60
9,UCSD,7.302632,6.039474,0.811404,0.671053,1.182408,0.994897,NaN,NaN


## Matchup Expected Runs Matrix

In [5]:
#Duplicate above function with team parameter
def find_expected_runs(team, opponent):
    if team == opponent or opponent == 'Average' or team == 'Average':
        return np.nan
    
    att = runs_master.loc[runs_master['Team'] == team, 'Conf ATT'].iloc[0]
    defense = runs_master.loc[runs_master['Team'] == opponent, 'Conf DEF'].iloc[0]
    
    xruns = (att * defense) * average_runs * np.sqrt(1 + hfa[team])
    return round(xruns, 2)

# Build full matchup matrix
teams = runs_master.loc[runs_master['Team'] != 'Average', 'Team'].tolist()
matchup_matrix = pd.DataFrame(index=teams, columns=teams)

for team in teams:
    for opp in teams:
        matchup_matrix.loc[team, opp] = find_expected_runs(team, opp)

# Save to Excel
matchup_matrix.to_excel("full_matchup_matrix_bigwest.xlsx")

matchup_matrix


,Cal Bap,Cal Poly,Bakersfield,Fullerton,CSUN,LBSU,Sacramento State,UCI,UCR,UCSD,UCSB,Utah Valley
Cal Bap,NaN,5.69,6.87,6.57,8.68,7.31,5.69,6.08,9.05,6.82,5.63,6.97
Cal Poly,7.92,NaN,7.86,7.52,9.93,8.36,6.51,6.96,10.35,7.8,6.44,7.97
Bakersfield,4.75,3.9,NaN,4.51,5.95,5.01,3.9,4.17,6.21,4.67,3.86,4.78
Fullerton,6.62,5.44,6.56,NaN,8.29,6.99,5.43,5.81,8.65,6.51,5.38,6.66
CSUN,6.91,5.68,6.85,6.55,NaN,7.29,5.67,6.06,9.03,6.8,5.62,6.95
LBSU,6.17,5.07,6.11,5.85,7.73,NaN,5.06,5.41,8.06,6.07,5.01,6.2
Sacramento State,5.69,4.67,5.64,5.4,7.13,6.0,NaN,4.99,7.43,5.6,4.62,5.72
UCI,9.21,7.57,9.14,8.74,11.54,9.73,7.57,NaN,12.04,9.07,7.49,9.27
UCR,4.67,3.84,4.63,4.43,5.85,4.93,3.84,4.1,NaN,4.6,3.8,4.7
UCSD,8.22,6.76,8.15,7.8,10.3,8.68,6.75,7.22,10.74,NaN,6.69,8.27


### Developing Probability Distribution

In [6]:
# Setting constants
m = -0.01219 
n = -1.1813
p = -0.3865
b = -1.042

# Setting 12 as the maximum runs/inning to consider
R_max = 12

In [7]:
def compute_C(A):
    numerator = A  # Expected total runs in game
    denominator = 9 * sum(
        (r * np.exp(m * r + p * r + n * (r / A) + b))
        for r in range(1, R_max + 1)
    ) 
    return numerator / denominator

def probability(A, R):
    C = compute_C(A)
    if R > 0:
        return C * np.exp(m * A + p * R + n * (R / A) + b)
    else:  # R == 0
        sum_term = sum(
            np.exp(m * A + p * r + n * (r / A) + b)
            for r in range(1, R_max + 1)
        )
        return 1 - C * sum_term

### Simulate single games using runs/inning probability distribution & season simulations

In [11]:
#Inning probability calculation 
def inning_probabilities(A):
    probs = [probability(A, r) for r in range(R_max + 1)]
    probs = np.array(probs)
    probs = probs / probs.sum()  # Normalize so sum = 1
    return probs

#Simulate one game 
def simulate_game(team_xruns, opp_xruns, innings=9):
    team_probs = inning_probabilities(team_xruns)
    opp_probs = inning_probabilities(opp_xruns)
    
    team_score = np.sum(np.random.choice(range(R_max+1), p=team_probs, size=innings))
    opp_score = np.sum(np.random.choice(range(R_max+1), p=opp_probs, size=innings))
    
    return team_score, opp_score

# Simulate full season for all matchups
def simulate_season_matchup_matrix(matchup_matrix, sims_per_game=100, filename="season_results.xlsx"):
    np.random.seed(42)
    results_summary = []

    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for team in matchup_matrix.index:
            team_results = []
            for opponent in matchup_matrix.columns:
                if team == opponent:
                    continue  # Skip matchups against self

                team_xruns = matchup_matrix.loc[team, opponent]
                opp_xruns = matchup_matrix.loc[opponent, team]

                if pd.isna(team_xruns) or pd.isna(opp_xruns):
                    continue

                # Simulate games
                game_results = [simulate_game(team_xruns, opp_xruns) for _ in range(sims_per_game)]
                games_df = pd.DataFrame(game_results, columns=[f"{team} Runs", f"{opponent} Runs"])
                games_df["Opponent"] = opponent

                # Append to this team's results
                team_results.append(games_df)

                # Calculate win percentage for summary
                win_pct = sum(1 for t, o in game_results if t > o) / sims_per_game
                results_summary.append((team, opponent, win_pct))

            # Combine all matchups for the team into one DataFrame
            if team_results:
                team_df = pd.concat(team_results, ignore_index=True)
                sheet_name = team[:31] 
                team_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Summary sheet
        summary_df = pd.DataFrame(results_summary, columns=["Team", "Opponent", "Win Percentage"])
        summary_df.to_excel(writer, sheet_name="Win Percentages", index=False)

    print(f"Full season simulation written to {filename}")
    
#simulate_season_matchup_matrix(matchup_matrix, sims_per_game=10000, filename="big_west_full_sim.xlsx")

In [12]:
winpcts = pd.read_excel('big_west_full_sim.xlsx', sheet_name = 'Win Percentages')
winpcts.head()

,Team,Opponent,Win Percentage
0,Cal Bap,Cal Poly,0.3377
1,Cal Bap,Bakersfield,0.5960
2,Cal Bap,Fullerton,0.4637
3,Cal Bap,CSUN,0.5574
4,Cal Bap,LBSU,0.5356


## Conference Expected Wins

In [13]:
winpcts['Expected Wins per Series'] = winpcts['Win Percentage'] * 3
winpcts

,Team,Opponent,Win Percentage,Expected Wins per Series
0,Cal Bap,Cal Poly,0.3377,1.0131
1,Cal Bap,Bakersfield,0.5960,1.7880
2,Cal Bap,Fullerton,0.4637,1.3911
3,Cal Bap,CSUN,0.5574,1.6722
4,Cal Bap,LBSU,0.5356,1.6068
...,...,...,...,...
127,Utah Valley,Sacramento State,0.4702,1.4106
128,Utah Valley,UCI,0.3121,0.9363
129,Utah Valley,UCR,0.7258,2.1774
130,Utah Valley,UCSD,0.4026,1.2078


In [14]:
def expected_wins_choose_k(df, k, team):
    results = []
    team_df = df[df['Team'] == team]  # Remove self 
    for combo in itertools.combinations(range(len(team_df)), k):
        total_wins = team_df.iloc[list(combo)]['Expected Wins per Series'].sum() / (k * 3)
        results.append(total_wins)
    return sorted(results)

def simulate_all_teams_wl(winpcts, filename):
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for team in winpcts['Team'].unique():
            projected_winning_pct = {}

            # 33 games (11 series)
            wrate_33 = expected_wins_choose_k(winpcts, 11, team)[0]
            w_33 = wrate_33 * 33
            l_33 = 33 - w_33
            wl_33 = f'{round(w_33, 1)} - {round(l_33, 1)}'
            projected_winning_pct[33] = {
                'W': w_33, 'L': l_33, 'Win%': wrate_33, 'W-L': wl_33
            }

            # 30 games (10 series)
            wrate_30_min = np.min(expected_wins_choose_k(winpcts, 10, team))
            wrate_30_max = np.max(expected_wins_choose_k(winpcts, 10, team))
            projected_winning_pct[30] = {
                'Min': {
                    'W': wrate_30_min * 30,
                    'L': 30 - wrate_30_min * 30,
                    'Win%': wrate_30_min,
                    'W-L': f'{round(wrate_30_min*30, 0)} - {round(30-wrate_30_min*30, 0)}'
                },
                'Max': {
                    'W': wrate_30_max * 30,
                    'L': 30 - wrate_30_max * 30,
                    'Win%': wrate_30_max,
                    'W-L': f'{round(wrate_30_max*30, 0)} - {round(30-wrate_30_max*30, 0)}'
                }
            }

            # 27 games (9 series)
            wrate_27_min = np.min(expected_wins_choose_k(winpcts, 9, team))
            wrate_27_max = np.max(expected_wins_choose_k(winpcts, 9, team))
            projected_winning_pct[27] = {
                'Min': {
                    'W': wrate_27_min * 27,
                    'L': 27 - wrate_27_min * 27,
                    'Win%': wrate_27_min,
                    'W-L': f'{round(wrate_27_min*27, 0)} - {round(27-wrate_27_min*27, 0)}'
                },
                'Max': {
                    'W': wrate_27_max * 27,
                    'L': 27 - wrate_27_max * 27,
                    'Win%': wrate_27_max,
                    'W-L': f'{round(wrate_27_max*27, 0)} - {round(27-wrate_27_max*27, 0)}'
                }
            }

            # 24 games (8 series)
            wrate_24_min = np.min(expected_wins_choose_k(winpcts, 8, team))
            wrate_24_max = np.max(expected_wins_choose_k(winpcts, 8, team))
            wrate_24_med = np.median(expected_wins_choose_k(winpcts, 8, team))
            projected_winning_pct[24] = {
                'Min': {
                    'W': wrate_24_min * 24,
                    'L': 24 - wrate_24_min * 24,
                    'Win%': wrate_24_min,
                    'W-L': f'{round(wrate_24_min*24, 0)} - {round(24-wrate_24_min*24, 0)}'
                },
                'Max': {
                    'W': wrate_24_max * 24,
                    'L': 24 - wrate_24_max * 24,
                    'Win%': wrate_24_max,
                    'W-L': f'{round(wrate_24_max*24, 0)} - {round(24-wrate_24_max*24, 0)}'
                },
                'Median': {
                    'W': wrate_24_med * 24,
                    'L': 24 - wrate_24_med * 24,
                    'Win%': wrate_24_med,
                    'W-L': f'{round(wrate_24_med*24, 0)} - {round(24-wrate_24_med*24, 0)}' 
                }
            }

            # Build DataFrame
            games = []
            for g_count, values in projected_winning_pct.items():
                if all(k in values for k in ['W', 'L', 'Win%', 'W-L']):
                    games.append({'Games': g_count, 'Type': 'Point', **values})
                else:
                    for bound, vals in values.items():
                        games.append({'Games': g_count, 'Type': bound, **vals})

            df_team = pd.DataFrame(games)
            df_team.to_excel(writer, sheet_name=team[:31], index=False)  # Limit to 31 chars

    print(f"Projected W-L records written to {filename}")

#simulate_all_teams_wl(winpcts, filename="projected_win_loss_by_team.xlsx")

In [15]:
#Example
projection_utah = pd.read_excel('projected_win_loss_by_team.xlsx', sheet_name='Utah Valley')
projection_utah

,Games,Type,W,L,Win%,W-L
0,33,Point,15.7107,17.2893,0.476082,15.7 - 17.3
1,30,Min,13.5333,16.4667,0.451110,14.0 - 16.0
2,30,Max,14.7744,15.2256,0.492480,15.0 - 15.0
3,27,Min,11.7372,15.2628,0.434711,12.0 - 15.0
4,27,Max,13.7490,13.2510,0.509222,14.0 - 13.0
5,24,Min,10.0458,13.9542,0.418575,10.0 - 14.0
6,24,Max,12.7017,11.2983,0.529237,13.0 - 11.0
7,24,Median,11.4696,12.5304,0.477900,11.0 - 13.0


## Non-Conference Simulations

In [16]:
nonconf = pd.read_csv('files/2025 Non-Conference Game to Game Results - Sheet1.csv')
nonconf2025 = nonconf[['Date', 'away_team', 'away_score', 'home_team', 'home_score', 'innings']]

def big_west_nonconference_records(df):
    
    conference_teams = [
        'UC Riverside\n Highlanders', 
        'CS Fullerton\n Titans', 
        'UC San Diego\n Tritons', 
        'UC Irvine\n Anteaters', 
        'Utah Valley\n Wolverines',
        'Long Beach St\n 49ers', 
        'UC Santa Barbara\n Gauchos', 
        'CS Bakersfield\n Roadrunners', 
        'CS Northridge\n Matadors', 
        'Cal Poly\n Mustangs', 
        'Cal Baptist\n Lancers', 
        'CS Sacramento\n Hornets']
    
    conference_teams_apprev = {
        'UC Riverside\n Highlanders': 'UCR', 
        'CS Fullerton\n Titans': 'Fullerton', 
        'UC San Diego\n Tritons': 'UCSD', 
        'UC Irvine\n Anteaters': 'UCI', 
        'Utah Valley\n Wolverines': 'Utah Valley',
        'Long Beach St\n 49ers': 'LBSU', 
        'UC Santa Barbara\n Gauchos': 'UCSB', 
        'CS Bakersfield\n Roadrunners': 'Bakersfield', 
        'CS Northridge\n Matadors': 'CSUN', 
        'Cal Poly\n Mustangs': 'Cal Poly', 
        'Cal Baptist\n Lancers': 'Cal Bap',
        'CS Sacramento\n Hornets': 'Sacramento State'}
    
    nonconf_record = []
    
    for team in conference_teams:
        # Get games where team is home or away
        team_df = df[(df['away_team'] == team) | (df['home_team'] == team)]
        
        # Exclude games against other conference teams
        team_df = team_df[
            ~((team_df['away_team'].isin(conference_teams)) & (team_df['home_team'].isin(conference_teams)))
        ]
        
        # Away games: win if away_score > home_score
        away_df = team_df[team_df['away_team'] == team].copy()
        away_df['W'] = away_df['away_score'] > away_df['home_score']
        
        # Home games: win if home_score > away_score
        home_df = team_df[team_df['home_team'] == team].copy()
        home_df['W'] = home_df['home_score'] > home_df['away_score']
        
        # Combine and sort
        winloss = pd.concat([away_df, home_df], ignore_index=True).sort_values(by='Date')
        
        wins = winloss['W'].sum()
        losses = len(winloss) - wins
        winpct = round(wins / len(winloss), 2)
        
        apprev = conference_teams_apprev[team]
        xcl = pd.read_excel('projected_win_loss_by_team.xlsx', sheet_name=apprev)
        confw = xcl['W'].iloc[7]
        confl = xcl['L'].iloc[7]
        
        ovr_w = confw + wins
        ovr_l = confl + losses
        over_wpct = round(ovr_w / (ovr_w + ovr_l), 3)
        
        
        nonconf_record.append({
            'Team': team, 
            'Non-Conference Win Percentage': winpct, 
            'Conference Wins': confw, 
            'Conference Losses': confl, 
            'Non-Conference Wins': wins, 
            'Non-Conference Losses': losses, 
            'Overall Wins': ovr_w, 
            'Overall Losses': ovr_l,
            'Overall Win Percentage': over_wpct})
        
    result = pd.DataFrame(nonconf_record)
    result['Conference Simulated Win Percentage'] = [0.216754545, 0.470254545, 0.549954545, 0.647081818, 0.476081818, 0.397572727, 0.6024, 0.3236, 0.373109091, 0.6058, 0.468354545, 0.463945455]
        
    return result

nonconfwinpct = big_west_nonconference_records(nonconf2025)
nonconfwinpct.to_csv('big west projected win percentages.csv')
nonconfwinpct

,Team,Non-Conference Win Percentage,Conference Wins,Conference Losses,Non-Conference Wins,Non-Conference Losses,Overall Wins,Overall Losses,Overall Win Percentage,Conference Simulated Win Percentage
0,UC Riverside\n Highlanders,0.44,5.2035,18.7965,8,10,13.2035,28.7965,0.314,0.216755
1,CS Fullerton\n Titans,0.47,11.3316,12.6684,14,16,25.3316,28.6684,0.469,0.470255
2,UC San Diego\n Tritons,0.48,13.2717,10.7283,12,13,25.2717,23.7283,0.516,0.549955
3,UC Irvine\n Anteaters,0.68,15.5175,8.4825,21,10,36.5175,18.4825,0.664,0.647082
4,Utah Valley\n Wolverines,0.52,11.4696,12.5304,28,26,39.4696,38.5304,0.506,0.476082
5,Long Beach St\n 49ers,0.37,9.5715,14.4285,10,17,19.5715,31.4285,0.384,0.397573
6,UC Santa Barbara\n Gauchos,0.78,14.4501,9.5499,21,6,35.4501,15.5499,0.695,0.602400
7,CS Bakersfield\n Roadrunners,0.31,7.8018,16.1982,10,22,17.8018,38.1982,0.318,0.323600
8,CS Northridge\n Matadors,0.28,8.9931,15.0069,7,18,15.9931,33.0069,0.326,0.373109
9,Cal Poly\n Mustangs,0.64,14.5221,9.4779,21,12,35.5221,21.4779,0.623,0.605800


## Adding RPI + non-conference record into simulation

In [17]:
final_records = pd.read_csv('files/big west projected win percentages.csv')
final_records['Team'] = final_records['Team'].replace({'UC San Diego\n Tritons': 'UCSD',
                                           'CS Northridge\n Matadors': 'CSUN',
                                           'Pacific\nTigers' : 'Pacific',
                                           'Santa Clara\nBroncos': 'Santa Clara',
                                           'San Diego St\nAztecs': 'SDSU',
                                           'UC Santa Barbara\n Gauchos': 'UCSB',
                                           'Oregon St\nBeavers': 'OSU',
                                           'UC Irvine\n Anteaters': 'UCI',
                                           'Fresno St\nBulldogs': 'Fresno St',
                                           'San Diego\nToreros': 'USD',
                                           'Arizona St\nSun Devils': 'ASU',
                                           'San Francisco\nDons': 'San Francisco',
                                           'San Jose St\nSpartans': 'SJSU',
                                           'Loy Marymount\nLions': 'LMU',
                                           'USC\nTrojans': 'USC',
                                           'Utah Tech\nRebels': 'Utah Tech',
                                           'Nevada\nWolf Pack': 'Nevada',
                                           'Pepperdine\nWaves': 'Pepperdine',
                                           'UC Riverside\n Highlanders': 'UCR',
                                           'Cal Baptist\n Lancers': 'Cal Bap',
                                           'Cal Poly\n Mustangs': 'Cal Poly',
                                           'Long Beach St\n 49ers': 'LBSU',
                                           'CS Sacramento\n Hornets': 'Sacramento State',
                                           'UCLA\nBruins': 'UCLA',
                                           'CS Bakersfield\n Roadrunners': 'Bakersfield',
                                           'CS Fullerton\n Titans': 'Fullerton',
                                           'Utah Valley\n Wolverines': 'Utah Valley'
                                           }, regex=False)

final_records

,Unnamed: 0,Team,Non-Conference Win Percentage,Conference Wins,Conference Losses,Non-Conference Wins,Non-Conference Losses,Overall Wins,Overall Losses,Overall Win Percentage,Conference Simulated Win Percentage
0,0,UCR,0.44,7.1529,25.8471,8,10,15.1529,35.8471,0.297,0.216755
1,1,Fullerton,0.47,15.5184,17.4816,14,16,29.5184,33.4816,0.469,0.470255
2,2,UCSD,0.48,18.1485,14.8515,12,13,30.1485,27.8515,0.520,0.549955
3,3,UCI,0.68,21.3537,11.6463,21,10,42.3537,21.6463,0.662,0.647082
4,4,Utah Valley,0.52,15.7107,17.2893,28,26,43.7107,43.2893,0.502,0.476082
5,5,LBSU,0.37,13.1199,19.8801,10,17,23.1199,36.8801,0.385,0.397573
6,6,UCSB,0.78,19.8792,13.1208,21,6,40.8792,19.1208,0.681,0.602400
7,7,Bakersfield,0.31,10.6788,22.3212,10,22,20.6788,44.3212,0.318,0.323600
8,8,CSUN,0.28,12.3126,20.6874,7,18,19.3126,38.6874,0.333,0.373109
9,9,Cal Poly,0.64,19.9914,13.0086,21,12,40.9914,25.0086,0.621,0.605800


In [48]:
import pandas as pd
import itertools

def expected_wins_choose_k(winpcts, k, team, final_records):
    # Only rows where the target team is playing (long-form)
    team_df = winpcts[winpcts['Team'] == team].reset_index(drop=True)
    
    results = []
    
    # Generate all combinations of k opponents
    for combo_indexes in itertools.combinations(range(len(team_df)), k):
        combo_df = team_df.iloc[list(combo_indexes)]
        
        # Expected win% for the team across these series
        total_wins = combo_df['Expected Wins per Series'].sum() / (k * 3)  # divide by total games
        
        # Opponent average overall win% from final_records
        opps = combo_df['Opponent'].tolist()
        opp_win_pct = final_records[final_records['Team'].isin(opps)]['Overall Win Percentage'].mean()
        
        # Compute RPI: 25% team win + 75% opponent strength
        rpi = 0.25 * total_wins + 0.75 * opp_win_pct
        
        # Store results
        results.append((total_wins, opp_win_pct, rpi))
    
    # Sort by team's win% (ascending)
    return sorted(results, key=lambda x: x[0])


def simulate_ucsd_with_rpi(winpcts, final_records, filename):
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        projected_winning_pct = {}

        def add_case(num_games, num_series):
            combos = expected_wins_choose_k(winpcts, num_series, 'UCSD', final_records)
            min_case = combos[0]        # lowest win%
            max_case = combos[-1]       # highest win%
            median_case = combos[len(combos) // 2]  # middle element
            
            projected_winning_pct[num_games] = {
                'Min': {
                    'W': min_case[0] * num_games,
                    'L': num_games - min_case[0] * num_games,
                    'Win%': min_case[0],
                    'RPI': min_case[2],
                    'W-L': f'{round(min_case[0]*num_games, 0)} - {round(num_games-min_case[0]*num_games, 0)}'
                },
                'Median': {
                    'W': median_case[0] * num_games,
                    'L': num_games - median_case[0] * num_games,
                    'Win%': median_case[0],
                    'RPI': median_case[2],
                    'W-L': f'{round(median_case[0]*num_games, 0)} - {round(num_games-median_case[0]*num_games, 0)}'
                },
                'Max': {
                    'W': max_case[0] * num_games,
                    'L': num_games - max_case[0] * num_games,
                    'Win%': max_case[0],
                    'RPI': max_case[2],
                    'W-L': f'{round(max_case[0]*num_games, 0)} - {round(num_games-max_case[0]*num_games, 0)}'
                }
            }

        # Run for each scenario
        add_case(33, 11)
        add_case(30, 10)
        add_case(27, 9)
        add_case(24, 8)

        # Convert to DataFrame
        games = []
        for g_count, values in projected_winning_pct.items():
            for bound, vals in values.items():
                games.append({'Games': g_count, 'Type': bound, **vals})

        df_team = pd.DataFrame(games)
        df_team.to_excel(writer, sheet_name='UCSD', index=False)

    print(f"Projected W-L + RPI for UCSD written to {filename}")

    
simulate_ucsd_with_rpi(winpcts, final_records, 'projected win-loss and rpi.xlsx')

Projected W-L + RPI for UCSD written to projected win-loss and rpi.xlsx


In [18]:
nonconf = pd.read_excel('files/2025 Big West Conference Runs.xlsx', sheet_name = 'Non-Conference Game Log')
nonconf

,Date,away_team,away_score,home_team,home_score,innings,Unnamed: 6,total_teams,unique_teams,num_appearances,Notes,valid_nonconf_teams
0,2025-02-14 00:00:00,Gonzaga\nBulldogs,6.0,UC Davis\nAggies,1.0,9.0,NaN,UC Davis\nAggies,UC Davis\nAggies,141.0,CONFERENCE,Pacific
1,2025-02-14 00:00:00,Utah Valley\nWolverines,2.0,UC Riverside\nHighlanders,3.0,9.0,NaN,UC Riverside\nHighlanders,UC Riverside\nHighlanders,125.0,CONFERENCE,Santa Clara
2,2025-02-14 00:00:00,Cal Poly\nMustangs,2.0,UCLA\nBruins,3.0,9.0,NaN,UCLA\nBruins,UCLA\nBruins,21.0,NOTED,LMU
3,2025-02-14 00:00:00,Lamar\nCardinals,17.0,CS Bakersfield\nRoadrunners,1.0,9.0,NaN,CS Bakersfield\nRoadrunners,CS Bakersfield\nRoadrunners,135.0,CONFERENCE,USC
4,2025-02-14 00:00:00,Campbell\nCamels,1.0,UC Santa Barbara\nGauchos,6.0,9.0,NaN,UC Santa Barbara\nGauchos,UC Santa Barbara\nGauchos,145.0,CONFERENCE,Fresno St
...,...,...,...,...,...,...,...,...,...,...,...,...
2151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UC Irvine\nAnteaters,NaN,NaN,NaN,NaN
2152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Long Beach St\n49ers,NaN,NaN,NaN,NaN
2153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Texas A&M\nAggies,NaN,NaN,NaN,NaN
2154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,San Jose St\nSpartans,NaN,NaN,NaN,NaN


In [19]:
#Identify potential non-conference teams with a large amount of games played against Big West opponents
potential_nonconf_teams = list(nonconf['valid_nonconf_teams'].unique())
potential_nonconf_teams = potential_nonconf_teams[:-1].copy()
potential_nonconf_teams


['Pacific',
 'Santa Clara',
 'LMU',
 'USC',
 'Fresno St',
 'Utah Tech',
 'Nevada',
 'Pepperdine',
 'SDSU',
 'USD',
 'SJSU',
 'ASU',
 'San Francisco',
 'Oregon State']

In [20]:
potential_nonconf_teams = [
    'Pacific\nTigers', 
    'Santa Clara\nBroncos', 
    'San Francisco\nDons', 
    'San Jose St\nSpartans', 
    'Loy Marymount\nLions', 
    'USC\nTrojans',
    'Fresno St\nBulldogs',
    'Utah Tech\nRebels',
    'Nevada\nWolf Pack',
    'Pepperdine\nWaves',
    'San Diego St\nAztecs',
    'San Diego\nToreros',
    'Arizona St\nSun Devils',
    'San Francisco\nDons',
    'Oregon St\nBeavers',
    'UCLA\nBruins'
]
    

In [21]:
nonconf_gamelog = nonconf[(nonconf['away_team'].isin(potential_nonconf_teams) | nonconf['home_team'].isin(potential_nonconf_teams))][['Date', 'away_team', 'away_score', 'home_team', 'home_score', 'innings']].copy().reset_index(drop=True)
nonconf_gamelog.sample(5)

,Date,away_team,away_score,home_team,home_score,innings
262,2025-06-01 00:00:00,UC Irvine\nAnteaters,3.0,Oregon St\nBeavers,5.0,9.0
250,2025-05-07 00:00:00,UC San Diego\nTritons,8.0,San Diego\nToreros,9.0,9.0
231,2025-04-23 00:00:00,USC\nTrojans,6.0,CS Fullerton\nTitans,4.0,9.0
56,2025-03-12 00:00:00,Pacific\nTigers,11.0,CS Bakersfield\nRoadrunners,10.0,9.0
2,2025-02-14 00:00:00,CS Northridge\nMatadors,4.0,Santa Clara\nBroncos,5.0,9.0


In [22]:
nonconf_gamelog = nonconf_gamelog.replace({'UC San Diego\nTritons': 'UCSD',
                                           'CS Northridge\nMatadors': 'CSUN',
                                           'Pacific\nTigers' : 'Pacific',
                                           'Santa Clara\nBroncos': 'Santa Clara',
                                           'San Diego St\nAztecs': 'SDSU',
                                           'UC Santa Barbara\nGauchos': 'UCSB',
                                           'Oregon St\nBeavers': 'OSU',
                                           'UC Irvine\nAnteaters': 'UCI',
                                           'Fresno St\nBulldogs': 'Fresno St',
                                           'San Diego\nToreros': 'USD',
                                           'Arizona St\nSun Devils': 'ASU',
                                           'San Francisco\nDons': 'San Francisco',
                                           'San Jose St\nSpartans': 'SJSU',
                                           'Loy Marymount\nLions': 'LMU',
                                           'USC\nTrojans': 'USC',
                                           'Utah Tech\nRebels': 'Utah Tech',
                                           'Nevada\nWolf Pack': 'Nevada',
                                           'Pepperdine\nWaves': 'Pepperdine',
                                           'UC Riverside\nHighlanders': 'UCR',
                                           'Cal Baptist\nLancers': 'Cal Bap',
                                           'Cal Poly\nMustangs': 'Cal Poly',
                                           'Long Beach St\n49ers': 'LBSU',
                                           'CS Sacramento\nHornets': 'Sacramento State',
                                           'UCLA\nBruins': 'UCLA',
                                           'CS Bakersfield\nRoadrunners': 'Bakersfield',
                                           'CS Fullerton\nTitans': 'Fullerton',
                                           'Utah Valley\nWolverines': 'Utah Valley'
                                           }, regex=False)

new_nonconf = nonconf_gamelog[(~((nonconf_gamelog['away_team'] == 'UC Davis\nAggies') | (nonconf_gamelog['home_team'] == 'UC Davis\nAggies'))) & (~((nonconf_gamelog['away_team'] == 'Hawaii\nRainbow Warriors') | (nonconf_gamelog['home_team'] == 'Hawaii\nRainbow Warriors')))].reset_index(drop=True)
new_nonconf

,Date,away_team,away_score,home_team,home_score,innings
0,2025-02-14 00:00:00,Cal Poly,2.0,UCLA,3.0,9.0
1,2025-02-14 00:00:00,UCSD,1.0,Pacific,2.0,9.0
2,2025-02-14 00:00:00,CSUN,4.0,Santa Clara,5.0,9.0
3,2025-02-15 00:00:00,Cal Poly,2.0,UCLA,18.0,9.0
4,2025-02-15 00:00:00,UCSD,11.0,Pacific,4.0,9.0
...,...,...,...,...,...,...
319,2025-05-09 00:00:00,UCI,10.0,USD,2.0,9.0
320,2025-05-09 00:00:00,Fullerton,2.0,UCLA,10.0,9.0
321,2025-05-16 00:00:00,UCSD,5.0,SDSU,3.0,9.0
322,2025-05-16 00:00:00,Fresno St,6.0,Cal Poly,11.0,9.0


In [23]:
new_nonconf.to_csv('files/2025 Non-Conference Game Log.csv', index=False)

In [24]:
nonconf_runs = pd.DataFrame({
    'Team': ['Pacific','Santa Clara', 'San Francisco', 'UCLA', 'USC', 'LMU', 'OSU', 'ASU', 'SJSU', 'SDSU', 'USD', 'Pepperdine', 'Nevada', 'Utah Tech', 'Fresno St', 'UCSD'],
    'R/GP': [4.133333333, 6.294117647, 6.333333333, 6.75, 6.730769231, 5.933333333, 8.333333333, 9.8, 4.416666667, 5.5, 5.153846154, 4.657142857, 7.076923077, 5.625, 6.125, 6],
    'RA/GP': [6.266666667, 4.882352941, 5.166666667, 5.583333333, 4.692307692, 6.766666667, 4.888888889, 7.2, 9.416666667, 6.533333333, 6.730769231, 8.485714286, 8, 5.9375, 7.46875, 6.108],
    'NCATT': [0.668940532, 1.01864284, 1.024989525, 1.092423046, 1.08931073, 0.960253344, 1.348670427, 1.586036423, 0.714795327, 0.890122482, 0.834100787, 0.753714102, 1.145332425, 0.910352538, 0.991272764, 0.971042708],
    'NCDEF': [0.962919983, 0.750209874, 0.793896794, 0.857920729, 0.721008007, 1.039748705, 0.751214171, 1.106333597, 1.446940932, 1.003895301, 1.034232797, 1.303893168, 1.229259553, 0.912341074, 1.147629035, 0.93855628]
})

nonconf_runs

,Team,R/GP,RA/GP,NCATT,NCDEF
0,Pacific,4.133333,6.266667,0.668941,0.962920
1,Santa Clara,6.294118,4.882353,1.018643,0.750210
2,San Francisco,6.333333,5.166667,1.024990,0.793897
3,UCLA,6.750000,5.583333,1.092423,0.857921
4,USC,6.730769,4.692308,1.089311,0.721008
5,LMU,5.933333,6.766667,0.960253,1.039749
6,OSU,8.333333,4.888889,1.348670,0.751214
7,ASU,9.800000,7.200000,1.586036,1.106334
8,SJSU,4.416667,9.416667,0.714795,1.446941
9,SDSU,5.500000,6.533333,0.890122,1.003895


In [25]:
nonconf_hfa = {
    'Pacific': 0.26,
    'Santa Clara': 0.23,
    'San Francisco': 0.26,
    'UCLA': 0.22,
    'USC': 0.22,
    'LMU': 0.17,
    'OSU': 0.21,
    'ASU': 0.26,
    'SJSU': 0.27,
    'SDSU': 0.29,
    'USD': 0.21,
    'Pepperdine': 0.23,
    'Nevada': 0.25,
    'Utah Tech': 0.22,
    'Fresno St': 0.23,
    'UCSD': 0.24
}

In [26]:
average_runs = nonconf_runs['R/GP'].mean()

def find_expected_runs(opponent):
    
    if opponent == 'UCSD': 
        return np.nan
    
    #Find our offensive rating
    att = nonconf_runs[nonconf_runs['Team'] == 'UCSD']['NCATT'].iloc[0]
    
    #Find opponents defensive rating
    defense = nonconf_runs[nonconf_runs['Team'] == opponent]['NCDEF'].iloc[0]
    
    #Calculate expected runs
    xruns = (att * defense) * average_runs * np.sqrt(1 + nonconf_hfa['UCSD'])
    
    return round(xruns, 2)


def find_opp_expected_runs(opponent):
    
    if opponent == 'UCSD':
        return np.nan
    
    att = nonconf_runs[nonconf_runs['Team'] == opponent]['NCATT'].iloc[0]
    
    defense = nonconf_runs[nonconf_runs['Team'] == 'UCSD']['NCDEF'].iloc[0]
    
    xruns = (att * defense) * average_runs * np.sqrt(1 + nonconf_hfa[opponent])
    
    return round(xruns, 2)
    

nonconf_runs['ucsd_xruns'] = nonconf_runs['Team'].apply(find_expected_runs) 
nonconf_runs['x_runs'] = nonconf_runs['Team'].apply(find_opp_expected_runs)

nonconf_runs

,Team,R/GP,RA/GP,NCATT,NCDEF,ucsd_xruns,x_runs
0,Pacific,4.133333,6.266667,0.668941,0.962920,6.43,4.35
1,Santa Clara,6.294118,4.882353,1.018643,0.750210,5.01,6.55
2,San Francisco,6.333333,5.166667,1.024990,0.793897,5.30,6.67
3,UCLA,6.750000,5.583333,1.092423,0.857921,5.73,7.00
4,USC,6.730769,4.692308,1.089311,0.721008,4.82,6.98
5,LMU,5.933333,6.766667,0.960253,1.039749,6.95,6.02
6,OSU,8.333333,4.888889,1.348670,0.751214,5.02,8.60
7,ASU,9.800000,7.200000,1.586036,1.106334,7.39,10.32
8,SJSU,4.416667,9.416667,0.714795,1.446941,9.67,4.67
9,SDSU,5.500000,6.533333,0.890122,1.003895,6.71,5.86


In [27]:
#Duplicate above function with team parameter
def find_expected_runs(team, opponent):
    if team == opponent or opponent == 'Average' or team == 'Average':
        return np.nan
    
    att = nonconf_runs.loc[nonconf_runs['Team'] == team, 'NCATT'].iloc[0]
    defense = nonconf_runs.loc[nonconf_runs['Team'] == opponent, 'NCDEF'].iloc[0]
    
    xruns = (att * defense) * average_runs * np.sqrt(1 + nonconf_hfa[team])
    return round(xruns, 2)

# Build full matchup matrix
teams = nonconf_runs.loc[nonconf_runs['Team'] != 'Average', 'Team'].tolist()
matchup_matrix = pd.DataFrame(index=teams, columns=teams)

for team in teams:
    for opp in teams:
        matchup_matrix.loc[team, opp] = find_expected_runs(team, opp)

# Save to Excel
matchup_matrix.to_excel("non_conference_matchup_matrix.xlsx")

nonconf_matrix = matchup_matrix

In [28]:
nonconf_matrix.head()

,Pacific,Santa Clara,San Francisco,UCLA,USC,LMU,OSU,ASU,SJSU,SDSU,USD,Pepperdine,Nevada,Utah Tech,Fresno St,UCSD
Pacific,NaN,3.48,3.68,3.98,3.35,4.82,3.49,5.13,6.71,4.66,4.8,6.05,5.7,4.23,5.32,4.35
Santa Clara,6.72,NaN,5.54,5.99,5.03,7.26,5.24,7.72,10.1,7.01,7.22,9.1,8.58,6.37,8.01,6.55
San Francisco,6.85,5.33,NaN,6.1,5.13,7.39,5.34,7.87,10.29,7.14,7.35,9.27,8.74,6.49,8.16,6.67
UCLA,7.18,5.59,5.92,NaN,5.38,7.75,5.6,8.25,10.79,7.48,7.71,9.72,9.16,6.8,8.56,7.0
USC,7.16,5.58,5.9,6.38,NaN,7.73,5.58,8.22,10.76,7.46,7.69,9.69,9.14,6.78,8.53,6.98


In [29]:
def inning_probabilities(A):
    probs = [probability(A, r) for r in range(R_max + 1)]
    probs = np.array(probs)
    probs = probs / probs.sum()  # Normalize so sum = 1
    return probs

#Simulate one game 
def simulate_game(team_xruns, opp_xruns, innings=9):
    team_probs = inning_probabilities(team_xruns)
    opp_probs = inning_probabilities(opp_xruns)
    
    team_score = np.sum(np.random.choice(range(R_max+1), p=team_probs, size=innings))
    opp_score = np.sum(np.random.choice(range(R_max+1), p=opp_probs, size=innings))
    
    return team_score, opp_score

# Simulate full season for all matchups
def simulate_season_matchup_matrix(matchup_matrix, sims_per_game=100, filename="season_results.xlsx"):
    np.random.seed(42)
    results_summary = []

    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for team in matchup_matrix.index:
            team_results = []
            for opponent in matchup_matrix.columns:
                if team == opponent:
                    continue  # Skip matchups against self

                team_xruns = matchup_matrix.loc[team, opponent]
                opp_xruns = matchup_matrix.loc[opponent, team]

                if pd.isna(team_xruns) or pd.isna(opp_xruns):
                    continue

                # Simulate games
                game_results = [simulate_game(team_xruns, opp_xruns) for _ in range(sims_per_game)]
                games_df = pd.DataFrame(game_results, columns=[f"{team} Runs", f"{opponent} Runs"])
                games_df["Opponent"] = opponent

                # Append to this team's results
                team_results.append(games_df)

                # Calculate win percentage for summary
                win_pct = sum(1 for t, o in game_results if t > o) / sims_per_game
                results_summary.append((team, opponent, win_pct))

            # Combine all matchups for the team into one DataFrame
            if team_results:
                team_df = pd.concat(team_results, ignore_index=True)
                sheet_name = team[:31] 
                team_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Summary sheet
        summary_df = pd.DataFrame(results_summary, columns=["Team", "Opponent", "Win Percentage"])
        summary_df.to_excel(writer, sheet_name="Win Percentages", index=False)

    print(f"Full season simulation written to {filename}")

In [31]:
simulate_season_matchup_matrix(nonconf_matrix, sims_per_game=10000, filename="non_conf_full_sim.xlsx")

In [ ]:
nonconf_winpct = pd.read_excel('non_conf_full_sim.xlsx', sheetname = 'Win Percentages')
ucsd_ncwinpct = nonconf_winpct[nonconf_winpct['Team'] == 'UCSD']
ucsd_ncwinpct

In [31]:
import pandas as pd
import numpy as np

def precompute_inning_probs(winpcts, R_max):
    """
    Precompute inning probability arrays for all unique expected runs.
    Returns a dictionary: {expected_runs: probability array}.
    """
    unique_xruns = np.unique(winpcts.values)
    probs_dict = {}
    for x in unique_xruns:
        probs = np.array([probability(x, r) for r in range(R_max+1)])
        probs_dict[x] = probs / probs.sum()  # normalize
    return probs_dict


def simulate_season_vectorized(matchup_matrix, R_max, sims_per_game=10000, filename="season_results.xlsx"):
    np.random.seed(42)
    results_summary = []
    
    # Precompute all probability distributions
    all_xruns = pd.concat([matchup_matrix, matchup_matrix.T]).stack().unique()
    probs_dict = {}
    for x in all_xruns:
        probs = np.array([probability(x, r) for r in range(R_max+1)])
        probs_dict[x] = probs / probs.sum()

    # Collect all team sheets in memory
    team_sheets = {}
    
    for team in matchup_matrix.index:
        team_results = []
        
        for opponent in matchup_matrix.columns:
            if team == opponent:
                continue

            team_xruns = matchup_matrix.loc[team, opponent]
            opp_xruns = matchup_matrix.loc[opponent, team]

            if pd.isna(team_xruns) or pd.isna(opp_xruns):
                continue

            # Vectorized simulation
            team_probs = probs_dict[team_xruns]
            opp_probs = probs_dict[opp_xruns]

            # Simulate innings all at once
            team_scores = np.random.choice(range(R_max+1), size=(sims_per_game, 9), p=team_probs)
            opp_scores  = np.random.choice(range(R_max+1), size=(sims_per_game, 9), p=opp_probs)

            team_total = team_scores.sum(axis=1)
            opp_total  = opp_scores.sum(axis=1)

            win_pct = np.mean(team_total > opp_total)
            results_summary.append((team, opponent, win_pct))

            # Prepare DataFrame for this matchup
            df = pd.DataFrame({
                f"{team} Runs": team_total,
                f"{opponent} Runs": opp_total,
                "Opponent": opponent
            })
            team_results.append(df)

        if team_results:
            team_sheets[team] = pd.concat(team_results, ignore_index=True)

    # Write all sheets at once
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for team, df in team_sheets.items():
            sheet_name = team[:31]  # Excel sheet name limit
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Summary sheet
        summary_df = pd.DataFrame(results_summary, columns=["Team", "Opponent", "Win Percentage"])
        summary_df.to_excel(writer, sheet_name="Win Percentages", index=False)

    print(f"Full season simulation written to {filename}")
    

simulate_season_vectorized(nonconf_matrix, R_max=12, sims_per_game=10000, filename="non_conf_full_sim.xlsx")

In [20]:
def simulate_season_custom_to_excel(df, filename, sims_per_game=100):
    np.random.seed(42)
    results = []
    
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for _, row in df.iterrows():
            team = row['Team']
            if team not in ['UCSD', 'Average']:
                ucsd_xruns = row['ucsd_xruns']
                opp_xruns = row['x_runs']

                # Simulate all games
                game_results = [simulate_game_custom(ucsd_xruns, opp_xruns) for _ in range(sims_per_game)]
                
                # Convert to DataFrame
                games_df = pd.DataFrame(game_results, columns=["UCSD Runs", f"{team} Runs"])
                
                # Write to Excel, each team gets its own sheet
                sheet_name = team[:31]  # Excel sheet names max length = 31
                games_df.to_excel(writer, sheet_name=sheet_name, index=False)
                
                game_results = [simulate_game_custom(ucsd_xruns, opp_xruns) for _ in range(sims_per_game)]
                win_pct = sum(1 for u, o in game_results if u > o) / sims_per_game
                results.append((team, win_pct))
                
        results_df = pd.DataFrame(results, columns=['Team', 'Win Percentage'])
                
        # Write the summary results to a separate sheet
        results_df.to_excel(writer, sheet_name='Win Percentages', index=False)

    print(f"Simulation results written to {filename}")

simulate_season_custom_to_excel(nonconf_runs, 'ucsd_nonconference_performance.xlsx', sims_per_game=10000)

Simulation results written to ucsd_nonconference_performance.xlsx


In [52]:
nonconf_proj = pd.read_excel('ucsd_nonconference_performance.xlsx', sheet_name = 'Win Percentages')
nonconf_proj['Opponent Wins'] = [20, 20, 26, 48, 37, 29, 48, 36, 29, 20, 28, 12, 34, 24, 31]
nonconf_proj['Opponent Losses'] = [36, 30, 30, 18, 23, 29, 16, 24, 30, 39, 30, 42, 23, 31, 29]
nonconf_proj['Winning Percentage'] = nonconf_proj['Opponent Wins'] / (nonconf_proj['Opponent Wins'] + nonconf_proj['Opponent Losses'])
nonconf_proj = nonconf_proj.rename(columns = {'Win Percentage': 'ucsd_xwp'})
nonconf_proj['Expected Wins per Series'] = nonconf_proj['ucsd_xwp'] * 3
nonconf_proj['Expected RPI'] = (0.25 * (nonconf_proj['Expected Wins per Series'] / 3)) + (0.75 * nonconf_proj['Winning Percentage'])
nonconf_proj.sort_values(by = 'Expected RPI', ascending = False)[['Team', 'Expected Wins per Series', 'Expected RPI']]


,Team,Expected Wins per Series,Expected RPI
3,UCLA,1.1397,0.640430
6,OSU,0.7920,0.628500
12,Nevada,1.5438,0.576018
8,SJSU,2.2083,0.552669
4,USC,0.9846,0.544550
7,ASU,0.9603,0.530025
14,Fresno St,1.6116,0.521800
5,LMU,1.5756,0.506300
10,USD,1.6728,0.501469
13,Utah Tech,1.4178,0.445423


In [47]:
def expected_wins_choose_k(winpcts, k):
    # Only rows where the target team is playing (long-form)
    team_df = winpcts
    
    results = []
    
    # Generate all combinations of k opponents
    for combo_indexes in itertools.combinations(range(len(team_df)), k):
        combo_df = team_df.iloc[list(combo_indexes)]
        
        # Expected win% for the team across these series
        total_wins = combo_df['Expected Wins per Series'].sum() / (k * 3)  # divide by total games
        
        # Opponent average overall win% from final_records
        opps = combo_df['Team'].tolist()
        opp_win_pct = team_df[team_df['Team'].isin(opps)]['Winning Percentage'].mean()
        
        # Compute RPI: 25% team win + 75% opponent strength
        rpi = 0.25 * total_wins + 0.75 * opp_win_pct
        
        results.append((total_wins, opp_win_pct, rpi))
    
    # Sort by team's win% (ascending)
    return sorted(results, key=lambda x: x[0])


def simulate_ucsd_with_rpi(winpcts, filename):
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        projected_winning_pct = {}

        def add_case(num_games, num_series):
            combos = expected_wins_choose_k(winpcts, num_series)
            min_case = combos[0]       # lowest win%
            max_case = combos[-1]      # highest win%
            median_case = combos[len(combos) // 2]  # middle value
            
            projected_winning_pct[num_games] = {
                'Min': {
                    'W': min_case[0] * num_games,
                    'L': num_games - min_case[0] * num_games,
                    'Win%': min_case[0],
                    'RPI': min_case[2],
                    'W-L': f'{round(min_case[0]*num_games, 0)} - {round(num_games-min_case[0]*num_games, 0)}'
                },
                'Median': {
                    'W': median_case[0] * num_games,
                    'L': num_games - median_case[0] * num_games,
                    'Win%': median_case[0],
                    'RPI': median_case[2],
                    'W-L': f'{round(median_case[0]*num_games, 0)} - {round(num_games-median_case[0]*num_games, 0)}'
                },
                'Max': {
                    'W': max_case[0] * num_games,
                    'L': num_games - max_case[0] * num_games,
                    'Win%': max_case[0],
                    'RPI': max_case[2],
                    'W-L': f'{round(max_case[0]*num_games, 0)} - {round(num_games-max_case[0]*num_games, 0)}'
                }
            }

        # Run for each scenario
        add_case(30, 10)
        add_case(27, 9)
        add_case(24, 8)
        add_case(21, 7)

        # Convert to DataFrame
        games = []
        for g_count, values in projected_winning_pct.items():
            for bound, vals in values.items():
                games.append({'Games': g_count, 'Type': bound, **vals})

        df_team = pd.DataFrame(games)
        df_team.to_excel(writer, sheet_name='UCSD', index=False)

    print(f"Projected W-L + RPI for UCSD written to {filename}")

    
simulate_ucsd_with_rpi(nonconf_proj, 'projected nonconf win-loss and rpi.xlsx')

Projected W-L + RPI for UCSD written to projected nonconf win-loss and rpi.xlsx
